In [ ]:
# ! pip install tiktoken
# ! pip install python-dotenv
# ! pip install pandas
# ! pip install pandasql
# ! pip install openai
# ! pip install langchain
# ! pip install langchain\[all\]
# ! pip install pyspark
# ! pip install findspark
# ! pip install faker

In [41]:
import os
import openai
import tiktoken
from dotenv import load_dotenv, find_dotenv

import pandas as pd
import warnings
import time
import random

warnings.filterwarnings('ignore')

### Define some questions to ask

In [ ]:
# question_1 = f"""Who has the longest average processing time?"""
# question_2 = f"""Who has the shortest average processing time?"""
# question_3 = f"""Who is the most productive participant based on the number of tasks completed?"""
# question_4 = f"""Who uses the least number of keystrokes on average?"""
# question_5 = f"""Name of the participant that uses the most number of long cut keys on average?""" # An invalid question, to check hallucination
# exception_question = f"""How was the day?"""

### Using LangChain for returning answers

In [42]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [43]:
import os
import yaml
import json

from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

from langchain.document_loaders import CSVLoader
from langchain.document_loaders.text import TextLoader

In [44]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [45]:
# GPT-4 works with Azure ChatOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.vectorstores import FAISS

#### Define Azure keys

In [46]:
api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_KEY")
api_type = 'azure'
model_deployment_name=os.getenv("AZURE_OPENAI_DNAME")
embeddings_deployment_name= os.getenv("AZURE_OPENAI_EMBED_NAME")
embedding_api_version='2022-12-01'
model_api_version='2023-05-15'

max_tokens = 1000
chat_temperature = 0
 #This will correspond to the custom name you chose for your deployment when you deployed a model. 

#### Read schema in JSON format
Why only JSON?

In [314]:
events_schema_file = '/Users/saishradhamohanty/Desktop/Repo/Prototypes/askskan/data/original/input_tables/pruned_events_schema.csv'
abstraction_instances_schema_file = '/Users/saishradhamohanty/Desktop/Repo/Prototypes/askskan/data/original/input_tables/abstraction_instances/schema.txt'
worktime_metrics_schema_file = '/Users/saishradhamohanty/Desktop/Repo/Prototypes/askskan/data/original/input_tables/worktime_metrics/schema.txt'

major_tables_metadata_file = '/Users/saishradhamohanty/Desktop/Repo/Prototypes/askskan/data/original/metadata/major_tables_metadata.yaml'

events_definitions_text_file = '/Users/saishradhamohanty/Desktop/Repo/Prototypes/askskan/data/original/input_tables/pruned_events_definitions.txt'
abstraction_instances_definitions_text_file = '/Users/saishradhamohanty/Desktop/Repo/Prototypes/askskan/data/original/input_tables/abstraction_instances/definitions.txt'
worktime_metrics_definitions_text_file = '/Users/saishradhamohanty/Desktop/Repo/Prototypes/askskan/data/original/input_tables/worktime_metrics/definitions.txt'

fake_data_file = '/Users/saishradhamohanty/Desktop/Repo/Prototypes/askskan/data/original/sample_data.csv'

#### Generate fake data based on original schema

In [ ]:
from faker import Faker
import pandas as pd

fake = Faker()

# Generate 10 unique rows of data
data = []
unique_values = set()

while len(data) < 20:
    row = {
        "event_id": fake.uuid4(),
        "sequence_id": fake.random_int(min=10000, max=99999),
        "event_time": fake.iso8601(),
        "persona_name": fake.name(),
        "app_name": fake.word(),
        "agent_type": fake.random_int(min=0, max=200),
        "clipboard": fake.random_int(min=0, max=10),
        "participant_name": fake.name(),
        "title": fake.sentence(),
        "event_date": fake.date(),
        "navigation_key_count": fake.random_int(min=0, max=10),
        "number_key_count": fake.random_int(min=0, max=10),
        "mouse_count": fake.random_int(min=0, max=10),
        "mouse_wheel": fake.random_int(min=0, max=10),
        "alpha_key_count": fake.random_int(min=0, max=10),
        "active_time": fake.random_int(min=0, max=100),
        "idle_time": random.uniform(0, 2000)/100.0, # fake.random_number(max=2000) / 100.0,
        "wait_time": random.uniform(0, 2000)/100.0,
        "processing_time": random.uniform(0, 1), # fake.random_number(max=1),
        "TaT_event": random.uniform(0, 1),
        "session_switch": fake.random_int(min=0, max=1),
        "app_switch": fake.random_int(min=0, max=1),
        "case_switch": fake.random_int(min=0, max=1),
        "activity_id": fake.uuid4(),
        "activity_abstraction_level_id": fake.uuid4(),
        "activity_abstraction_level_name": fake.word(),
        "parent_activity_id": fake.uuid4(),
        "activity_discovered_name": fake.sentence(),
        "activity_alias_name": fake.sentence(),
        "activity_instance_id": fake.uuid4(),
        "activity_instance_abstraction_level_alias_name": fake.word(),
        "activity_instance_original_end_time": fake.iso8601(),
        "activity_instance_end_time": fake.iso8601(),
        "activity_instance_event_count": fake.random_int(min=1, max=10),
        "activity_instance_start_time": fake.iso8601(),
        "case_id_name": fake.word(),
        "case_id_value": fake.word(),
        "url": fake.url(),
        "is_pruned": fake.random_int(min=0, max=1),
        "source": fake.random_element(["UA", "AA", "EA"]),
        "event_control_type": fake.random_int(min=10000, max=99999),
        "event_input_type": fake.random_int(min=100, max=999),
    }
    
    # Check if the generated row is unique
    row_tuple = tuple(row.values())
    if row_tuple not in unique_values:
        data.append(row)
        unique_values.add(row_tuple)

# Create a DataFrame from the data
df = pd.DataFrame(data)

# Print the dataset
# print(df.head())

# Convert DataFrame to CSV
df.to_csv('../data/original/sample_data.csv', index=False)



#### Convert csv to text file

In [ ]:
import csv

csv_file = definition_file
text_file = '../data/original/definitions.txt'

with open(csv_file, 'r') as file:
    reader = csv.DictReader(file)
    terms_definitions = []
    for row in reader:
        term = row['Term'].strip()
        definition = row['Definition'].strip()
        terms_definitions.append(f'{term}:{definition}')

with open(text_file, 'w') as file:
    file.write('\n'.join(terms_definitions))


#### Convert text to csv file

In [ ]:
import csv

# Specify the input text file path
input_file = file0

# Specify the output CSV file path
output_file = '../data/sample/schema0.csv'

# Define the field names for the CSV file
field_names = [
    'case_id',
    'case_status',
    'case_type',
    'task_name',
    'participant_id',
    'process_name',
    'process_variant',
    'applications_used',
    'processing_time',
    'wait_time',
    'turnaround_time',
    'start_time',
    'no_of_keystrokes',
    'no_of_shortcut_keys'
]

# Open the input file for reading
with open(input_file, 'r') as file:
    # Read the lines from the input file
    lines = file.readlines()

# Create a list to store the data rows for the CSV
data_rows = []

# Parse each line and extract the values
for line in lines:
    # Remove leading/trailing whitespaces and split the line by ':'
    line = line.strip()
    parts = line.split(':')

    # Extract the field name and value
    field_name = parts[0].strip()
    field_value = ':'.join(parts[1:]).strip()

    # Append the extracted values to the data row
    data_row = field_value
    data_rows.append(data_row)

# Write the data rows to the output CSV file
with open(output_file, 'w', newline='') as file:
    writer = csv.writer(file)
    
    # Write the header row
    writer.writerow(field_names)
    
    # Write the data rows
    writer.writerow(data_rows)


In [ ]:
# Read the Excel file and skip the first row
df = pd.read_excel(file2, skiprows=1)

# Drop the desired column
column_to_drop = ['Source', 'Comments']
df = df.drop(column_to_drop, axis=1)

# Convert DataFrame to CSV
df.to_csv('../data/original/schema1.csv', index=False)


#### Load documents that require embedding creation

In [291]:
loader_csv = CSVLoader(file_path=events_schema_file)
docs_csv = loader_csv.load()

#### Read other schema files in the text format

In [305]:
# Specify the path to the text file
file_path = abstraction_instances_schema_file

# Read the contents of the text file
with open(file_path, 'r') as file:
    abstraction_instances_schema = file.read()

In [302]:
# Specify the path to the text file
file_path = worktime_metrics_schema_file

# Read the contents of the text file
with open(file_path, 'r') as file:
    worktime_metrics_schema = file.read()

#### Read the metadata

In [292]:
import yaml

# Read YAML data from a file
input_yaml_file = major_tables_metadata_file
with open(input_yaml_file, 'r') as yaml_file:
    yaml_data = yaml.load(yaml_file, Loader=yaml.FullLoader)

print(yaml_data)

{'major tables': [{'name': 'unum_askskan.events_delta_tb', 'description': 'This table contains information related to events', 'usage': 'Use this table when asked about events for any persona', 'columns': ['event_id', 'event_time', 'sequence_id', 'persona_id', 'participant_id', 'application_id', 'agent_type', 'clipboard', 'title', 'event_date', 'navigation_key_count', 'number_key_count', 'mouse_count', 'mouse_wheel', 'alpha_key_count'], 'additional_minor_tables': ['describe_table_abstraction_instances', 'describe_worktime_metrics'], 'table_join_info': [{'join_set_1': [{'source_column': 'persona_id'}, {'target_column': 'persona_id'}, {'target_table': 'describe_table_abstraction_instances'}]}, {'join_set_2': [{'source_column': 'participant_id'}, {'target_column': 'participant_id'}, {'target_table': 'describe_worktime_metrics'}]}]}]}


In [293]:
# Read YAML data as text from a file
input_yaml_file = major_tables_metadata_file
with open(input_yaml_file, 'r') as yaml_file:
    table_metadata_text = yaml_file.read()

print(table_metadata_text)

major tables:
  - name: unum_askskan.events_delta_tb
    description: "This table contains information related to events"
    usage: "Use this table when asked about events for any persona"
    columns:
      - event_id
      - event_time
      - sequence_id
      - persona_id
      - participant_id
      - application_id
      - agent_type
      - clipboard
      - title
      - event_date
      - navigation_key_count
      - number_key_count
      - mouse_count
      - mouse_wheel
      - alpha_key_count
    additional_minor_tables:
      - describe_table_abstraction_instances
      - describe_worktime_metrics
    table_join_info:
      - join_set_1:
          - source_column: "persona_id"
          - target_column: "persona_id"
          - target_table: "describe_table_abstraction_instances"
      - join_set_2:
          - source_column: "participant_id"
          - target_column: "participant_id"
          - target_table: "describe_worktime_metrics"



#### Read the text files of definitions

In [295]:
# Specify the path to the text file
file_path = events_definitions_text_file

# Read the contents of the text file
with open(file_path, 'r') as file:
    events_definitions = file.read()

In [299]:
# Specify the path to the text file
file_path = abstraction_instances_definitions_text_file

# Read the contents of the text file
with open(file_path, 'r') as file:
    abstraction_instances_definitions = file.read()

In [297]:
# Specify the path to the text file
file_path = worktime_metrics_definitions_text_file

# Read the contents of the text file
with open(file_path, 'r') as file:
    worktime_metrics_definitions = file.read()

#### Split the documents into chunks

In [59]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7, chunk_overlap=6)
documents = text_splitter.split_documents(docs_csv)

In [60]:
embeddings = OpenAIEmbeddings(
                deployment=embeddings_deployment_name,
                openai_api_key=api_key,
                openai_api_base=api_base,
                openai_api_type=api_type,
                openai_api_version=embedding_api_version,
                chunk_size=16,
            )
print(embeddings)

vectorstore = FAISS.from_documents(documents, embeddings)


client=<class 'openai.api_resources.embedding.Embedding'> model='text-embedding-ada-002' deployment='architecture4I93pDgC2s-textemb' openai_api_version='2022-12-01' openai_api_base='https://skandevazopenai.openai.azure.com/' openai_api_type='azure' openai_proxy='' embedding_ctx_length=8191 openai_api_key='09ec6bb5136a4afab27de6a44fa4f998' openai_organization='' allowed_special=set() disallowed_special='all' chunk_size=16 max_retries=6 request_timeout=None headers=None


Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Embeddings_Create Operation under Azure OpenAI API version 2022-12-01 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 6 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Embeddings_Create Operation under Azure OpenAI API version 2022-12-01 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 2 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..


In [61]:
retriever = vectorstore.as_retriever()

#### Create template to provide as an input to the chain

In [25]:
zero_shot_table_prompt_v1 = """
You are an intelligent data analyst.

1. You are given table metadata delimited by <> in a YAML format that contains data about all the \
    available tables in the database.  
2. Every table entry under the tables section includes:
    - name: Identifies the table uniquely.
    - description: Provides a concise summary of the table's purpose and content.
    - usage: Offers insights into when one should utilize the information in the table.
    - columns: List of all the columns in the table.
3. For every table entry, information about how to join different possible tables join is specified in the table_join_info \
    section, information about different sets of joins is given in the join_set_i section, where i is the join set number.\
    For every join set, the following information is provided:
    - table_join_info: This field gives information about the columns that would be used for joining 2 different tables.\
        This is a list of tuples where:
            source_column: the first element of the tuple is the column in the current table that would be needed for joining with the \
                referenced table.
            target_column: the second element of the tuple is the column in the referenced table that would be needed for joining with the \
                current table.
            target_table: the third element is the name of the referenced table.
5. Strictly follow the following instructions:
    - You are given a question by the human delimited by <>.
    - Use the table metadata to identify the relevant tables in the database that would be needed to be \
        joined to answer the question based on the following conditions.
        - If the question can be answered using a single table then select only that table.
        - If the question needs information from multiple tables for answering the user question, then select the \
            minimum set of the relevant tables that can answer the question fully.
    - Return the selected table(s) in a list, refer to the example output below. Never include any extra text.
    - The table name should only be selected from the name field in the table metadata.
    - Omit any explanations.

Table metadata: <{table_metadata}>

Once the relevant table(s) are selected, use the following list format strictly to return the result, refer to the \ 
example output:
Output: ["events", "describe_table_abstraction_instances"]

Current conversation: 
Human: <{question}>
Table(s) selected: 
"""


In [228]:
zero_shot_table_prompt_v2 = """
You are an intelligent data analyst.

Table metadata: <{table_metadata}>

1. You are given table metadata delimited by <> in a YAML format that contains data about all the \
    available tables in the database.  
2. Every table entry under the major tables section includes:
    - name: Identifies the major table uniquely.
    - description: Provides a concise summary of the table's purpose and content.
    - usage: Offers insights into when one should utilize the information in the table.
    - columns: List of all the columns in the table.
    - additional_minor_tables: List of all the additional minor tables that should be joined with the current major table.
3. For every table entry, information about how to join different possible tables is specified in the table_join_info \
    section, information about different sets of joins is given in the join_set_i section, where i is the join set number.\
    For every join set, the following information is provided:
    - table_join_info: This field gives information about the columns that would be used for joining 2 different tables. The format is:
            source_column: the first element of the tuple is the column in the current major table that would be needed for joining with the \
                referenced table.
            target_column: the second element of the tuple is the column in the referenced minor table that would be needed \
                for joining with the current table.
            target_table: the third element is the name of the referenced minor table.
4. For obtaining the relevant tables strictly follow the following instruction step by step:
    - You are given a question by the human delimited by <>.
    - Return the selected table(s) as a list of list in the Output section using the steps listed below strictly:
        - First select the major table(s) that would be needed to answer the question.
        - Select the list of additional minor tables corresponding to the major table(s) selected.
        - You should create an inner list for each additional minor table of the selected major table irrespective of \
            whether the additional minor table is needed for answering the question or not.
        - For every minor table in the additional minor table list, you should create an inner list such that:
            - The major table for this minor table is the first element.
            - This minor table is the second element.
            - The column in the major table and the minor table that would be used for joining with the additional minor table. \
            Take this column from the target_column and source column fields under the table_join_info field in the table metadata.
        - Repeat the above step for each selected major table.
    _ Never assume anything on your own, just follow the above given instructions thoroughly.
    - Never include any extra text.
    - Omit any reasoning.

Following are examples of the user question along with the user question(s) and corresponding desired output. \
    Return the desired output by prepending the Output: keyword to the output as shown below.:
Question: "Which persona has the greatest worktime that is not whitelisted?"
Output: [["events", "describe_table_abstraction_instances", "persona_id"], ["events", "describe_worktime_metrics", "participant_id"]]
Reasoning: As the question needs information about persona, it will be obtained from the events table as the major table. \
            Since the instruction says to select the all of the additional minor tables corresponding to the major table, \
            hence, create an inner list for each additional minor table of the selected major table. \

Current conversation: 
Human: <{question}>
Table(s) selected: Output: 
"""


In [261]:
zero_shot_table_prompt_v3 = """
You are an intelligent data analyst.

Table metadata: <{table_metadata}>

1. You are given table metadata delimited by <> in a YAML format that contains data about all the \
    available tables in the database.  
2. Every table entry under the major tables section includes:
    - name: Identifies the major table uniquely.
    - description: Provides a concise summary of the table's purpose and content.
    - usage: Offers insights into when one should utilize the information in the table.
    - columns: List of all the columns in the table.
    - additional_minor_tables: List of all the additional minor tables that should be joined with the current major table.
3. For every table entry, information about how to join different possible tables is specified in the table_join_info \
    section, information about different sets of joins is given in the join_set_i section, where i is the join set number.\
    For every join set, the following information is provided:
    - table_join_info: This field gives information about the columns that would be used for joining 2 different tables. The format is:
            source_column: the first element of the tuple is the column in the current major table that would be needed for joining with the \
                referenced table.
            target_column: the second element of the tuple is the column in the referenced minor table that would be needed \
                for joining with the current table.
            target_table: the third element is the name of the referenced minor table.
4. For obtaining the relevant tables strictly follow the following instruction step by step:
    - You are given a question by the human delimited by <>.
    - Return the selected table(s) as a dictionary of a list of list in the Output section using the steps listed below strictly:
        - First select the major table(s) that would be needed to answer the question.
        - Select the list of additional minor tables corresponding to the major table(s) selected.
        - Create a dictionary with the major table(s) as the key and a list of list as the value. This list of list would be such that:
            - The number of inner lists would be equal to the number of additional minor tables corresponding to the major table.
            - For every minor table in the additional minor table list for the major table, you should create an inner list such that:
                - This minor table is the first element.
                - The column in the major table and the minor table that would be used for joining with the additional minor table as the third element. \
                Take this column from the target_column and source column fields under the table_join_info field in the table metadata.
        - Repeat the above step for each selected major table.
    _ Never assume anything on your own, just follow the above given instructions thoroughly.
    - Never include any extra text.
    - Omit any reasoning.

Following are examples of the user question along with the user question(s) and corresponding desired output. \
    Return the desired output by prepending the Output: keyword to the output as shown below.:
Question: "Which persona has the greatest worktime that is not whitelisted?"
Output: {{
            "unum_askskan.events_delta_tb": [["describe_table_abstraction_instances", "persona_id"], ["describe_worktime_metrics", "participant_id"]]
        }}
Reasoning: As the question needs information about persona, it will be obtained from the events table as the major table that forms a key of the \
            dictionary. Since the instruction says to select the all of the additional minor tables corresponding to the major table, \
            hence, create an inner list for each additional minor table of the selected major table. \

Current conversation: 
Human: <{question}>
Table(s) selected: Output: 
"""


In [307]:
query_generation_prompt_with_tables = """
You are a super smart code generator.
Perform the following actions:

1. Carefully understand the question by human, schema, schema definitions and context each delimited by <>.
2. Do not assume or create any schema attributes not given in the schema unless stated explicitly.
3. Do not assume any facts from the question unless stated explicitly.
4. Do not assume any values of the schema attributes that are not present in the schema or in the question. 
5. Strictly follow the following instructions:
    - The data table dictionary delimited by <> is a dictionary of list of list where:
        - The key is the name of the major table.
        - The value is a list of list where:
            - The first element is a minor table that needs to be joined with the major table.
            - The second element is the column in the major table and minor table that would be used for joining with the minor table.
    - The schema has names of the schema columns in Column Name, the descriptions of the columns in Description \
        and an example value of the schema column in Examples.
    - Use only the Column Name in the schema and the question to generate the final Spark SQL query from the tables in  \
        the data table dictionary for answering the original question. To join the tables, follow the instructions given below:
        - For every major table in the data table dictionary, join all the minor tables in the value list \
            of the major table to the major table.
        - Join the minor table with the major table using the column given along with the minor table in the value list of the major table.
    - Schema definitions and Description in schema can be referred for providing more clarity about Columns in the schema. \
        Strictly do not use values from schema definitions as column names or column values to generate the query.
    - The query result should be LIMITED by 10 rows.
    - The generated query should be from the start date to end date each delimited by <>. 
6. Output the correct Spark SQL query.
7. Select all the appropriate column(s) from the generated Spark SQL query by critically referring to the \
    requirements asked in the original question.
8. Write the answer in a friendly tone in response to the question.
9. Stricly use the following JSON schema format for the output, refer to the example output below. Never include any\
extra text.
10. Omit any explanations.

Events schema: <{context}>
Describe table abstraction instances schema: <{abstraction_instances_schema}>
Describe worktime metrics schema: <{worktime_metrics_schema}>
Events schema definitions: <{events_schema_definitions}>
Describe table abstraction instances schema definitions: <{abstraction_instances_schema_definitions}>
Describe worktime metrics schema definitions: <{worktime_metrics_schema_definitions}>
Data table dictionary: <{data_table_dictionary}>
Start date: <{start_date}>
End date: <{end_date}>


Once the Spark SQL query generated use only the following JSON schema format stricly, refer to the example output:
Output: {{
    "Query": The generated Spark SQL query from the start date to end date. Limit the results to maximum 10 rows for queries \
        with more than 1 row,
    "Column": The correct extracted column(s) from the Spark SQL query in a list,
    "Skan Bot": The final answer in a friendly tone. The answer from the SQL query should be delimited by ##.
}}

Following is an example of the output:
Output: {{
    "Query": "SELECT app_name, COUNT(*) AS count FROM clipboard WHERE event_date >= '2023-04-01' AND event_date <= '2023-04-30'" \
        GROUP BY app_name ORDER BY count DESC LIMIT 1',
    "Column": "app_name", 
    "Skan Bot": "The most used application is #result#."
}}

Output: {{
    "Query": "SELECT persona_name, SUM(worktime_whitelisted) AS total_whitelisted_worktime \
                FROM unum_askskan.events_delta_tb AS events_delta_tb \
                JOIN describe_worktime_metrics ON events_delta_tb.participant_id = describe_worktime_metrics.participant_id \
                WHERE event_date >= '2023-04-01' AND event_date <= '2023-04-30' \
                GROUP BY persona_name \
                ORDER BY total_whitelisted_worktime DESC \
                LIMIT 1",
    "Column": "persona_name", 
    "Skan Bot": "The persona with the highest whitelisted worktime is #result#."
}}

Please use "double quotes" for json keys and ensure the Output can be parsed by Python json.loads
    

Current conversation:
{chat_history}   
Human: <{question}>
Skan Bot: 
"""

#### Questions to be asked by the user

In [116]:
question_1 = "Which persona has the greatest worktime whitelisted?"
question_2 = "Total time spent of non process application?"
question_3 = "Is there an outstanding performer in CES persona?"
question_4 = "Which is the most used application?"
question_5 = "Which is the most active time window during the day?"
question_6 = "What is the average case effort per persona?"
question_7 = "What is the average utilization per persona?"
question_8 = "Which week had highest productivity in CES?"
question_9 = "What is the distribution of time spent in processing application per day?"
question_10 = "What is the average number of participants per case?"
question_11 = "What is the frequency of cases per participant per day?"
question_12 = "Who will be most productive next week?"
question_13 = "Any patterns observed that give an insight on inefficiencies?"
question_14 = "Would taking breaks improve the efficiency in performance?"

question_15 = "What is the standard deviation of time spent on process application?"
question_16 = "What is the standard deviation of time spent on non process application?"

# question_1 = f"""Who has the longest average processing time?"""
# question_2 = f"""Who has the shortest average processing time?"""
# question_3 = f"""Who is the most productive participant based on the number of tasks completed?"""
# question_4 = f"""Who uses the least number of keystrokes on average?"""
# question_5 = f"""Name of the participant that uses the most number of long cut keys on average?""" # An invalid question, to check hallucination
exception_question = f"""How was the day?"""

#### Providing output format

In [ ]:
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser

class FormatCodeOutput(BaseModel):
    Query: str = Field(description="The generated Spark SQL query")
    Code: str = Field(description="Python code generated to run the Spark SQL query")
    Skan_Bot: str = Field(description="The final answer printed by the python code in a friendly tone. Place the answer between ##")

In [ ]:
class FormatDoubtOutput(BaseModel):
    Doubt: str = Field(description="The clarification point asked by the bot")

#### Azure LLM

In [155]:
# Initialize LangChain with Azure OpenAI
chat_llm = AzureChatOpenAI(
    deployment_name=model_deployment_name,
    openai_api_version=model_api_version,
    openai_api_base=api_base,
    openai_api_key=api_key,
    temperature=0,
    streaming=True,
    verbose=True
)

In [335]:
from langchain.output_parsers import CommaSeparatedListOutputParser
table_selection_prompt = PromptTemplate(
    template=zero_shot_table_prompt_v3,
    input_variables=[
        "question",
    ],
    partial_variables={"table_metadata": table_metadata_text},
    output_parser=CommaSeparatedListOutputParser()
)


#### Add the first chain for obtaining relevant tables

In [336]:
table_selection_chain = LLMChain(prompt=table_selection_prompt, llm=chat_llm, verbose=False)
data_table_dict = table_selection_chain.predict(question="Which persona has the greatest whitelisted worktime?")
print(data_table_dict)

Output: {
            "unum_askskan.events_delta_tb": [["describe_table_abstraction_instances", "persona_id"], ["describe_worktime_metrics", "participant_id"]]
        }


In [337]:
string = data_table_dict

# Find and remove the first occurrence of "Query" in the string
stripped_output_section = string.replace(
                    'Output:', "").strip()
print(stripped_output_section)

{
            "unum_askskan.events_delta_tb": [["describe_table_abstraction_instances", "persona_id"], ["describe_worktime_metrics", "participant_id"]]
        }


In [338]:
import ast
# Using ast.literal_eval to safely evaluate the string as a literal Python expression
result_tables_dict = ast.literal_eval(stripped_output_section)
print(result_tables_dict)

{'unum_askskan.events_delta_tb': [['describe_table_abstraction_instances', 'persona_id'], ['describe_worktime_metrics', 'participant_id']]}


#### Add the 2nd chain for using tables and question etc to generate the query

In [339]:
query_generation_prompt = PromptTemplate(
    template=query_generation_prompt_with_tables,
    input_variables=[
        "context",
        "question",
        "chat_history",
        "data_table_dictionary"
    ],
    partial_variables={"events_schema_definitions": events_definitions,
                        "abstraction_instances_schema_definitions": abstraction_instances_definitions,
                        "worktime_metrics_schema_definitions": worktime_metrics_definitions_text_file,
                        "abstraction_instances_schema": abstraction_instances_schema,
                        "worktime_metrics_schema": worktime_metrics_schema,
                        "start_date": "2023-04-01",
                        "end_date": "2023-04-30"}
)

chain_type_kwargs = {"prompt": query_generation_prompt}

In [340]:
buffer_memory = ConversationBufferMemory(memory_key="chat_history", input_key="question")
query_generation_chain = ConversationalRetrievalChain.from_llm(
    llm = chat_llm, 
    retriever = retriever, 
    memory=buffer_memory,
    combine_docs_chain_kwargs=chain_type_kwargs,
    verbose=False,
    get_chat_history=lambda h:h
    )

In [341]:
query_generation_chain_input = {
    "question": "Which persona has the highest turnaround time?",
    "data_table_dictionary": result_tables_dict,
}


In [342]:
result = query_generation_chain(query_generation_chain_input)
print(result['answer'])

{
    "Query": "SELECT persona_name, AVG(turnaround_time) AS avg_turnaround_time FROM unum_askskan.events_delta_tb JOIN describe_table_abstraction_instances ON unum_askskan.events_delta_tb.persona_id = describe_table_abstraction_instances.persona_id WHERE abstraction_type = 'Activity' AND start_time >= '2023-04-01' AND end_time <= '2023-04-30' GROUP BY persona_name ORDER BY avg_turnaround_time DESC LIMIT 1",
    "Column": ["persona_name", "avg_turnaround_time"],
    "Skan Bot": "The persona with the highest turnaround time is #result#."
}


#### Obtaining results

In [ ]:
from langchain.output_parsers import RetryWithErrorOutputParser
retry_parser = RetryWithErrorOutputParser.from_llm(
    parser=parser, llm=OpenAI(temperature=0)
)

In [ ]:
retry_parser.parse_with_prompt(code, new_prompt)

AttributeError: 'PromptTemplate' object has no attribute 'to_string'

In [ ]:
result = qa({"question": "I want the output to be in another format."})
print(result['answer'])

Error in on_chain_start callback: 'name'


Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
Human: Which is the most used application?
AI: Doubt: Can you please clarify what you mean by "most used application"? Are you asking for the application with the highest active time or the application that was switched to the most frequently?
Human: Appliction with highest active time.
AI: Doubt: Are you asking for the application with the highest active time in general or for a specific time period?
Human: In general
AI: Output: The output should be formatted as a JSON instance that conforms to the JSON schema below.

```
{
  "Query": "SELECT application, MAX(active_time) AS highest_active_time FROM data_table GROUP BY application ORDER BY highest_active_time DESC LIMIT 1",
  "Code": "import pandas as pd\nfrom pyspark.sql import SparkSession\n\nspark = SparkSession.builder.getOrCreate()\ndata = pd.rea

Error in on_chain_start callback: 'name'
Error in on_chain_start callback: 'name'



> Finished chain.
Prompt after formatting:

You are a super smart code generator.
Perform the following actions:

1. Understand the input by human, schema and schema definitions each delimited by <>
2. Based on the question, the schema and the schema definitions, first seek to clarify any ambiguities.
    - Assume human does not know anything about the schema, so never ask details about the schema or schema         definitions or technical details in the clarification points.
    - First try to find the answer to the ambiguities from the schema and schema definitions yourself.
    - If the ambiguities still persist then: 
        1. Without mentioning about the schema or schema definitions only think of a list of super short bullets of             areas that need clarification in simple terms. 
        2. Then pick one clarification point, and wait for an answer from the human before moving to the next point.
3. Never ask the human to explain the schema or schema definitions.
4. Only 

In [ ]:
result = qa({"question": "I want the output to be in hours and minutes (HH:MM)."})
print(result['answer'])

Error in on_chain_start callback: 'name'


Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
Human: Which is the most used application?
AI: Doubt: Can you please clarify what you mean by "most used application"? Are you asking for the application with the highest active time or the application that was switched to the most frequently?
Human: Appliction with highest active time.
AI: Doubt: Are you asking for the application with the highest active time in general or for a specific time period?
Human: In general
AI: Output: The output should be formatted as a JSON instance that conforms to the JSON schema below.

```
{
  "Query": "SELECT application, MAX(active_time) AS highest_active_time FROM data_table GROUP BY application ORDER BY highest_active_time DESC LIMIT 1",
  "Code": "import pandas as pd\nfrom pyspark.sql import SparkSession\n\nspark = SparkSession.builder.getOrCreate()\ndata = pd.rea

Error in on_chain_start callback: 'name'
Error in on_chain_start callback: 'name'



> Finished chain.
Prompt after formatting:

You are a super smart code generator.
Perform the following actions:

1. Understand the input by human, schema and schema definitions each delimited by <>
2. Based on the question, the schema and the schema definitions, first seek to clarify any ambiguities.
    - Assume human does not know anything about the schema, so never ask details about the schema or schema         definitions or technical details in the clarification points.
    - First try to find the answer to the ambiguities from the schema and schema definitions yourself.
    - If the ambiguities still persist then: 
        1. Without mentioning about the schema or schema definitions only think of a list of super short bullets of             areas that need clarification in simple terms. 
        2. Then pick one clarification point, and wait for an answer from the human before moving to the next point.
3. Never ask the human to explain the schema or schema definitions.
4. Only 

In [ ]:
code = parser.parse(result['answer']).Code
print(code)

import pandas as pd
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
data = pd.read_csv('../askskan/data/original/sample_data.csv')
data_table = spark.createDataFrame(data)
data_table.createOrReplaceTempView('data_table')

query = "SELECT application, MAX(active_time) AS highest_active_time FROM data_table GROUP BY application ORDER BY highest_active_time DESC LIMIT 1"
result = spark.sql(query)
result.show()



In [ ]:
# result = qa({"question": "processing_time field represents the time taken by the total time taken by a participant for all their tasks"})
# print(result['answer'])

#### Clean the output string to look like JSON

In [ ]:
string = result['answer']

# Find and remove the first occurrence of "Query" in the string
stripped_string = string.replace('Output:', '', 1).strip()

print(stripped_string)


#### Extract the SQL Query

In [ ]:
import json

json_string = result['answer'] # stripped_string

# Remove invalid escape sequences from the JSON string
#json_string = json_string.encode('utf-8').decode('unicode_escape')

# Convert the modified JSON string to a JSON object
json_object = json.loads(json_string, strict=False)

# Extract the "Query" field
query = json_object['Query']

# Print the extracted query
print(query)


SELECT SUM(processing_time) FROM data_table WHERE agent_type = 0


#### Extract the Python code and answer string

In [ ]:
string = """
{"Query": ["SELECT SUM(processing_time) AS total_time FROM data_table WHERE agent_type != 0"],
"Code": ["import pandas as pd\nfrom pyspark.sql import SparkSession\n\nspark = \
    SparkSession.builder.appName('schema').getOrCreate()\ndata = spark.read.csv('data/sample/data0.csv', \
        header=True, inferSchema=True)\ndata.createOrReplaceTempView('data_table')\n\nquery = \
            "SELECT SUM(processing_time) AS total_time FROM data_table WHERE agent_type != 0"\nresult = \
                spark.sql(query)\nresult.show()\nprint('The total time spent on process applications is:', \
                    result.collect()[0][0])"],
"Skan Bot": ["The total time spent on process applications is: #total_time#."],
}"""

In [ ]:
string1 = """
{"Code": "import pandas as pd\nfrom pyspark.sql import SparkSession\n\nspark = \
SparkSession.builder.appName('schema').getOrCreate()\ndata = spark.read.csv('data/sample/data0.csv', \
header=True, inferSchema=True)\ndata.createOrReplaceTempView('data_table')\n\nquery = \
'SELECT SUM(processing_time) AS total_time FROM data_table WHERE agent_type != 0'\nresult = \
spark.sql(query)\nresult.show()\nprint('The total time spent on process applications is:', \
result.collect()[0][0])"
}"""

In [ ]:
import json

json_string = string1 # result['answer']

# Remove invalid escape sequences from the JSON string
json_string = json_string.encode('utf-8').decode('unicode_escape')

# Find and remove the first occurrence of "Query" in the JSON string
#json_string = json_string.replace('Query', '', 1).strip()

# Convert the modified JSON string to a JSON object
json_object = json.loads(json_string, strict=False)

# Extract the "Query" field
code = json_object['Code']
#answer = json_object['Skan Bot']

# Print the extracted query
print(code)
#print(answer)

import pandas as pd
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('schema').getOrCreate()
data = spark.read.csv('data/sample/data0.csv', header=True, inferSchema=True)
data.createOrReplaceTempView('data_table')

query = 'SELECT SUM(processing_time) AS total_time FROM data_table WHERE agent_type != 0'
result = spark.sql(query)
result.show()
print('The total time spent on process applications is:', result.collect()[0][0])


In [ ]:
try:
    exec(code)
except Exception as e:
    from pyspark.sql import SparkSession
    from pyspark.sql.functions import sum

    # Create a SparkSession
    spark = SparkSession.builder.getOrCreate()

    # Load the 'data' table into a DataFrame
    data = spark.table('data')

    # Apply the filter condition
    filtered_data = data.filter(data.agent_type != 0)

    # Calculate the sum of processing_time
    result = filtered_data.select(sum('processing_time').alias('total_time'))

    # Show the result
    result.show()


In [ ]:
import findspark
findspark.init() 

from pyspark.sql import SparkSession

from pyspark import SparkConf, SparkContext

#spark = SparkSession.builder.appName('Query').getOrCreate()

# conf = SparkConf().setAppName("YourAppName").setMaster("local").set("spark.driver.bindAddress", "127.0.0.1")
# spark = SparkContext(conf=conf)
spark = SparkSession.builder.appName("YourAppName").master("local").config("spark.driver.bindAddress", "127.0.0.1").getOrCreate()

data = spark.read.format('csv').option('header', 'true').load('../data/sample/data0.csv')
data.createOrReplaceTempView('data0')

query = 'SELECT AVG(processing_time) AS avg_processing_time FROM data'
avg_processing_time = spark.sql(query).collect()[0]['avg_processing_time']

print('The longest average processing time overall is: {} seconds.'.format(avg_processing_time))

In [ ]:
import pandas as pd
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Query').getOrCreate()
data = spark.read.format('csv').option('header', 'true').load('../data/sample/data0.csv')
data.createOrReplaceTempView('data0')

query = 'SELECT participant_id, AVG(processing_time) AS avg_processing_time \
FROM data0 GROUP BY participant_id \
ORDER BY avg_processing_time \
DESC LIMIT 1'

result_df = spark.sql(query).toPandas()
print('The participant with the longest average processing time is:', result_df['participant_id'][0])

In [ ]:
exec(code)

In [ ]:
from azure.identity import DefaultAzureCredential 
from azure.storage.blob import BlobClient

# Storage Account name
account_name = os.environ['DATALAKE_ACCOUNT_NAME']
# Storage Account key
account_key = os.environ['DATALAKE_ACCOUNT_KEY']

# Name of the container where the blob is stored
container_name = "model-training"

# Name of the blob you want to fetch
blob_name = "DataShare/data1/2023-03-01-to-2023-03-31/events/events.csv"

file_path = "../data/original/"
file_name = "events.csv"


account_url = "https://{account_key}.blob.core.windows.net"
# Create a BlobClient object with data transfer options for download
blob_client = BlobClient(
    account_url=account_url, 
    container_name=container_name, 
    blob_name=blob_name,
    credential=account_key, #DefaultAzureCredential(),
    max_single_get_size=1024*1024*32, # 32 MiB
    max_chunk_get_size=1024*1024*4 # 4 MiB
)

with open(file=os.path.join(r'../askskan/data/original/', 'events.csv'), mode="wb") as sample_blob:
    download_stream = blob_client.download_blob(max_concurrency=2)
    sample_blob.write(download_stream.readall())

ServiceRequestError: <urllib3.connection.HTTPSConnection object at 0x140512820>: Failed to resolve 'model-training' ([Errno 8] nodename nor servname provided, or not known)